In [ ]:
%pip install path_vis

In [ ]:
from path_vis import PathVis
import numpy as np

lst = np.concatenate([
      [1, 0, 0, 0.0, 0, 1, 0, 0.0, 0, 0, 1, 0.0, 0, 0, 0, 1],
      [1, 0, 0, 0.2, 0, 1, 0, 0.1, 0, 0, 1, 0.0, 0, 0, 0, 1],
      [1, 0, 0, 0.4, 0, 1, 0, 0.15, 0, 0, 1, 0.1, 0, 0, 0, 1],
]).tolist()

PathVis(lst)

In [ ]:
import numpy as np

def rotz(phi):
    c, s = np.cos(phi), np.sin(phi)
    R = np.eye(4)
    R[:2, :2] = [[c, -s], [s, c]]
    return R

def roty(theta):
    c, s = np.cos(theta), np.sin(theta)
    R = np.eye(4)
    R[0, 0], R[0, 2] = c, s
    R[2, 0], R[2, 2] = -s, c
    return R

def segment_matrix(k, phi, s):
    if abs(k) < 1e-9:
        seg = np.eye(4)
        seg[2, 3] = s
    else:
        theta, r = k * s, 1 / k
        seg = roty(theta)
        seg[0, 3] = r * (1 - np.cos(theta))
        seg[2, 3] = r * np.sin(theta)
        seg = rotz(phi) @ seg @ rotz(-phi)
    return seg

rng = np.random.default_rng()
kappa = rng.uniform(0,20,3)
l = rng.uniform(0.1,0.3,3)
phi = rng.uniform(0, 2*np.pi, 3)

curr_frame = np.eye(4)
n = 10
curve = [ curr_frame.copy() ]

for ki, li, pi in zip(kappa, l, phi):
    for j in range(1, n + 1):
        seg = curr_frame @ segment_matrix(ki, pi, j * li / n)
        curve.append(seg)
    curr_frame @= segment_matrix(ki, pi, li)

flat_curve = np.concatenate([c.ravel() for c in curve]).tolist()

from path_vis import PathVis
PathVis(flat_curve)
